# INF8225 - DPA - Anomaly Detection







## Import & Git

In [ ]:
# Import git projet

!git clone https://github.com/deca1111/Anomaly_Detection_Comparison.git

# changement de folder
%cd /content/Anomaly_Detection_Comparison/

Cloning into 'INF8225-Projet'...
remote: Enumerating objects: 2168, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 2168 (delta 56), reused 108 (delta 37), pack-reused 2041
Receiving objects: 100% (2168/2168), 475.30 MiB | 67.66 MiB/s, done.
Resolving deltas: 100% (714/714), done.
/content/INF8225-Projet


In [ ]:
# Instalation modules utiles
# ! pip install -r requirements.txt

! pip install tensorboardX
! pip install pyyaml
! pip install patool
!pip install setuptools>=41.4.0
!pip install pretrainedmodels>=0.7.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 2.6 MB/s eta 0:00:00


In [ ]:
# Import utiles

from anomaly_detection.dpa.train import Trainer
from anomaly_detection.dpa.evaluate import evaluate

import torch
import torch.nn as nn

import numpy as np
import yaml

import time
import os
import sys

import urllib.request
import zipfile
import patoolib

import pandas as pd

import argparse

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(DEVICE)

cuda


# Import Dataset

## Unzip Dataset

In [ ]:
compressedFile = "/content/Anomaly_Detection_Comparison/data/data/nih/subset_300_zip/images_subset_300.zip"

targetImagesRoot = "/content/Anomaly_Detection_Comparison/data/data/nih_300/"

# Création folder
os.makedirs(targetImagesRoot, exist_ok=True)

# Unzip
patoolib.extract_archive(compressedFile, outdir=targetImagesRoot)

INFO patool: Extracting /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip ...
INFO:patool:Extracting /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip ...
INFO patool: running /usr/bin/7z x -o/content/INF8225-Projet/data/data/nih_300/ -- /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip
INFO:patool:running /usr/bin/7z x -o/content/INF8225-Projet/data/data/nih_300/ -- /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip extracted to `/content/INF8225-Projet/data/data/nih_300/'.
INFO:patool:... /content/INF8225-Projet/data/data/nih/subset_300_zip/images_subset_300.zip extracted to `/content/INF8225-Projet/data/data/nih_300/'.


'/content/INF8225-Projet/data/data/nih_300/'

## Vérification du Subset

In [ ]:
listImagesFile = "/content/Anomaly_Detection_Comparison/data/data/nih/images_subset_300.txt"

rootImages = "/content/Anomaly_Detection_Comparison/data/data/nih_300/"

# vérification que l'intégralité des images du subset est présent

with open(listImagesFile, 'r') as f:
    listImages = f.read().splitlines()

imagesManquantes = []
nbImages = 0
for image in listImages:
    if not os.path.exists(rootImages + image):
        imagesManquantes.append(image)
    else :
        nbImages += 1

if len(imagesManquantes) > 0:
  print(f"Images manquantes : {imagesManquantes}")
else:
  print(f"Tous les images du subset sont présentes [{nbImages}/{len(listImages)}]")

Tous les images du subset sont présentes [7311/7311]


# Entrainement et évaluation

In [ ]:
# Config piad
model = "dpa"
dataset = "nih"
exp = "wo_pg"
_class = "subset"


RUNS = ['0', '1', '2']

configs = []

for run in RUNS:

  config = f"configs/{model}/{dataset}/final/{exp}/class_{_class}/run_{run}/train_eval.yaml"

  print(config)
  configs.append(config)

def _load_config(path):
    with open(path, 'r') as stream:
        config = yaml.load(stream, Loader=yaml.FullLoader)
    return config

configs/dpa/nih/final/wo_pg/class_subset/run_0/train_eval.yaml
configs/dpa/nih/final/wo_pg/class_subset/run_1/train_eval.yaml
configs/dpa/nih/final/wo_pg/class_subset/run_2/train_eval.yaml


## Run 0

### Training

In [ ]:
start_time = time.time()
trainer = Trainer(_load_config(configs[0]))
trainer.train()
print(f"Training is complete. Took: {(time.time() - start_time) / 60:.02f}m")
del trainer

adam_kwargs:
  lr: 0.002
batch_sizes:
  64: 128
checkpoint_root: ./data/checkpoint/dpa/nih/final/wo_pg/class_subset/run_0
dec:
  kwargs:
    inner_dims:
    - 16
    - 16
    - 16
    - 16
    - 16
    - 16
    - 16
  type: residual18
early_stoping_patience: 1000000
early_stopping_min_delta: 0.001
early_stopping_patience: 3
enc:
  kwargs:
    inner_dims:
    - 16
    - 16
    - 16
    - 16
    - 16
    - 16
    - 16
  type: residual18
finetune_from: null
image_dim: 1
image_rec_loss:
  loss_kwargs:
    use_smooth_pg: false
    weights_per_resolution:
      64:
        feature_weights:
          r42: 1
        img_weight: 0
  loss_type: relative_perceptual_L1
image_sample_iter: 1000
initial_image_res: 64
iters_per_res:
  64:
    stab: 16000
latent_dim: 16
latent_res: 1
log_iter: 10
log_root: ./data/logs/dpa/nih/final/wo_pg/class_subset/run_0
max_image_res: 64
random_seed: 813
results_root: ./data/results/dpa/nih/final/wo_pg/class_subset/run_0
stab_iter: 100000
test_batch_size: 32
test_da

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


================================== MODELS ON LARGEST RESOLUTION ==================================
====================== Encoder =============================
STABNetwork(
  (model): Sequential(
    (preprocess_res_64): Sequential(
      (0): ConvBlock(
        (model): Sequential(
          (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (1): PreActResnetBlock(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): ConvBlock(
            (model): Sequential(
              (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): LeakyReLU(negative_slope=0.2)
            )
          )
          (2): ConvBlock(
            (model): Sequential(
              (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
          )
        )
        (skipcon): Sequential()
      )
      (2): PreActResnetBlock(
        (model): Sequential(
          (0): L

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:04<00:00, 143MB/s]
  0%|          | 0/16000 [00:00<?, ?it/s]

====================== Encoder =============================
STABNetwork(
  (model): Sequential(
    (preprocess_res_64): Sequential(
      (0): ConvBlock(
        (model): Sequential(
          (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        )
      )
      (1): PreActResnetBlock(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): ConvBlock(
            (model): Sequential(
              (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): LeakyReLU(negative_slope=0.2)
            )
          )
          (2): ConvBlock(
            (model): Sequential(
              (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
          )
        )
        (skipcon): Sequential()
      )
      (2): PreActResnetBlock(
        (model): Sequential(
          (0): LeakyReLU(negative_slope=0.2)
          (1): ConvBlock(
            (model): Sequential(
           

 10%|▉         | 1532/16000 [06:48<1:01:23,  3.93it/s]

### Évaluation

In [ ]:
start_time = time.time()

evaluate(_load_config(configs[0]))

print(f"Evaluation is complete. Took: {(time.time() - start_time) / 60:.02f}m")

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Save model and results
import shutil

# Define the paths for the original folder and the destination folder
original_folder_path_1 = '/content/Anomaly_Detection_Comparison/data/checkpoint'
original_folder_path_2 = '/content/Anomaly_Detection_Comparison/data/logs'
original_folder_path_3 = '/content/Anomaly_Detection_Comparison/data/results'
destination_folder_path_1 = '/content/drive/MyDrive/UdeM/INF8225-AI/Project/Results/woPG/checkpoint'
destination_folder_path_2 = '/content/drive/MyDrive/UdeM/INF8225-AI/Project/Results/woPG/logs'
destination_folder_path_3 = '/content/drive/MyDrive/UdeM/INF8225-AI/Project/Results/woPG/results'


# Copy the entire folder and its contents to the destination
shutil.copytree(original_folder_path_1, destination_folder_path_1)
shutil.copytree(original_folder_path_2, destination_folder_path_2)
shutil.copytree(original_folder_path_3, destination_folder_path_3)